<a href="https://colab.research.google.com/github/kKn00077/techit-travel-data-analysis/blob/main/japan_txt_youtube.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install deepl

In [ ]:
import os
from google.colab import userdata
import googleapiclient.discovery as discovery
import googleapiclient.errors as errors

api_key = userdata.get('google_api_key')
youtube = discovery.build('youtube', 'v3', developerKey=api_key)

# keywords = ['韓国旅行', '韓国地方旅行', '韓国 ソウル', '韓国 プサン', '韓国のお祭り'] # , '韓国 コスメ'

keywords = ['韓国旅行', '韓国旅行 韓国のお祭り']
video_ids = []

for keyword in keywords:
    request = youtube.search().list(
        part="id",
        maxResults=25,
        q=keyword
    )
    response = request.execute()

    # videoId만 추출하여 리스트로 저장
    video_ids = video_ids + [{ 'keyword': keyword ,'video_ids': item['id']['videoId']} for item in response['items'] if 'videoId' in item['id']]


# video_ids = list(set(video_ids))
print(video_ids)


[{'keyword': '韓国旅行', 'video_ids': 'w9-1fQk9AS0'}, {'keyword': '韓国旅行', 'video_ids': 'hG7TEoeblvs'}, {'keyword': '韓国旅行', 'video_ids': '9Fcg0UUtTd8'}, {'keyword': '韓国旅行', 'video_ids': 'g5CHkaIip-w'}, {'keyword': '韓国旅行', 'video_ids': 'Sn4-K27TZSc'}, {'keyword': '韓国旅行', 'video_ids': 'XLDVJbgBBDw'}, {'keyword': '韓国旅行', 'video_ids': 'UAn81Dw6UWw'}, {'keyword': '韓国旅行', 'video_ids': 'JefIzbAFxag'}, {'keyword': '韓国旅行', 'video_ids': 'jlomygnoR_w'}, {'keyword': '韓国旅行', 'video_ids': 'SPNozUDD1Ms'}, {'keyword': '韓国旅行', 'video_ids': 'fIt8tDSqzLw'}, {'keyword': '韓国旅行', 'video_ids': '00TluTLeYFs'}, {'keyword': '韓国旅行', 'video_ids': 'bcw9WYKc1uA'}, {'keyword': '韓国旅行', 'video_ids': 'ynO6YqHDPMI'}, {'keyword': '韓国旅行', 'video_ids': 'dfpt8CwUQ6w'}, {'keyword': '韓国旅行', 'video_ids': 'fVYcLb5BvHw'}, {'keyword': '韓国旅行', 'video_ids': 'O35g4st43ks'}, {'keyword': '韓国旅行', 'video_ids': 'JLvZP-aOrKg'}, {'keyword': '韓国旅行', 'video_ids': '4hlbKGHpzXs'}, {'keyword': '韓国旅行', 'video_ids': 'cQCaFJ2vtwk'}, {'keyword': '韓国旅行',

In [ ]:
import re

# 댓글 가져오기 함수
def get_comments(youtube, video_id):
    comments = []
    years = []
    next_page_token = None

    while True:
        try:
            request = youtube.commentThreads().list(
                part="snippet",
                videoId=video_id,
                pageToken=next_page_token,
                maxResults=100,  # 한 번에 최대 100개의 댓글을 가져옴
                textFormat="plainText"
            )
            response = request.execute()

            for item in response['items']:
                year = item['snippet']['topLevelComment']['snippet']['publishedAt'][:4]

                if year == '2023' or year == '2024':
                    comment = item['snippet']['topLevelComment']['snippet']['textDisplay']
                    comments.append(comment)
                    years.append(year)

            # 다음 페이지가 있으면 계속해서 가져옴
            next_page_token = response.get('nextPageToken')
            if not next_page_token:
                break
        except errors.HttpError as e:
            if e.resp.status == 403:  # 댓글이 비활성화된 경우
                print(f"댓글이 비활성화된 동영상입니다: {video_id}")
                break
            else:
                raise e
                break # 다른 오류는 그대로 발생시킴

    return comments, years

# 간단한 텍스트 전처리 함수
def preprocess_text(text):
    text = text.lower()  # 소문자로 변환
    text = re.sub(r'\s+', ' ', text)  # 공백 정리
    text = re.sub(r'[^\w\s]', '', text)  # 특수 문자 제거
    return text

# 여러 동영상에 대한 댓글 수집 및 텍스트 전처리
all_comments = []
all_years = []
all_keywords = []

for video in video_ids:
    video_id = video['video_ids']
    keyword = video['keyword']
    print(f"Fetching comments for video ID: {video_id}")
    comments, years = get_comments(youtube, video_id)  # 동영상별로 댓글 가져오기
    processed_comments = [preprocess_text(comment) for comment in comments]  # 전처리
    all_keywords.extend(([keyword] * len(processed_comments)))
    all_comments.extend(processed_comments)  # 모든 댓글을 한 리스트에 저장
    all_years.extend(years)  # 모든 댓글의 연도 리스트에 저장

Fetching comments for video ID: w9-1fQk9AS0
Fetching comments for video ID: hG7TEoeblvs
Fetching comments for video ID: 9Fcg0UUtTd8
Fetching comments for video ID: g5CHkaIip-w
Fetching comments for video ID: Sn4-K27TZSc
Fetching comments for video ID: XLDVJbgBBDw
Fetching comments for video ID: UAn81Dw6UWw
Fetching comments for video ID: JefIzbAFxag
Fetching comments for video ID: jlomygnoR_w
Fetching comments for video ID: SPNozUDD1Ms
Fetching comments for video ID: fIt8tDSqzLw
Fetching comments for video ID: 00TluTLeYFs
Fetching comments for video ID: bcw9WYKc1uA
Fetching comments for video ID: ynO6YqHDPMI
Fetching comments for video ID: dfpt8CwUQ6w
Fetching comments for video ID: fVYcLb5BvHw
Fetching comments for video ID: O35g4st43ks
Fetching comments for video ID: JLvZP-aOrKg
Fetching comments for video ID: 4hlbKGHpzXs
Fetching comments for video ID: cQCaFJ2vtwk
Fetching comments for video ID: M1uelmf39b4
Fetching comments for video ID: y-1T_KQFZlE
Fetching comments for video ID: 

Fetching comments for video ID: qL1v6Rk5Bfg
Fetching comments for video ID: nIG0S5aJZ1Y
Fetching comments for video ID: ap4fa1K486o
댓글이 비활성화된 동영상입니다: ap4fa1K486o
Fetching comments for video ID: xdRFeoDJGu8
Fetching comments for video ID: S_Pn-eo3dFU
Fetching comments for video ID: vyu_TzoAqoM
Fetching comments for video ID: HGIKZw46ne4
Fetching comments for video ID: XLDVJbgBBDw


In [ ]:
print(len(all_comments))
print(len(all_years))
print(len(all_keywords))

9597
9597
9597


In [ ]:
# 유튜브 댓글 export csv

import pandas as pd

df = pd.DataFrame({'keywords' : all_keywords, 'comments' : all_comments, 'years' : all_years})
display(df.shape)

# 중복제거
df_cleaned = df.drop_duplicates()

# 최대한 일본인의 댓글만 가져오기 위해 한글이 포함된 행 제거
df_cleaned = df_cleaned[~df_cleaned["comments"].str.contains(r"[가-힣]", na=False)]  # "col1" 기준으로 필터링

display(df_cleaned.shape)

(9597, 3)

(9063, 3)

In [ ]:
from google.cloud import translate_v2 as translate
import os

# 서비스 계정 키 파일 경로 설정
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/content/google-cloud-api-key2.json"

# 번역 클라이언트 생성
client = translate.Client()

def translate_text(text):
    try:
        translated = client.translate(text, source_language="ja", target_language="ko")
        return translated['translatedText']
    except Exception as e:
        print(f"Error translating text: {text}. Error: {e}")
        return 'error'

df_cleaned["trans_comments"] = df_cleaned["comments"].apply(translate_text)
display(df_cleaned.head())

,keywords,comments,years,trans_comments
0,韓国旅行,wifiは韓国の日本語サイトで予約したら半額で借りれる,2024,wifi는 한국의 일본어 사이트에서 예약하면 반액으로 빌릴 수 있습니다.
1,韓国旅行,辛くないが辛い辛くないの食べたい時はどうしたらいいんだ,2024,힘들지 않지만 괴롭지 않지만 먹고 싶을 때는 어떻게해야합니까?
3,韓国旅行,韓国も随分変わりましたな昔は英語は日本より通じなくて手書きの漢字の方が中年の方との会話には通...,2024,한국도 상당히 바뀌었구나 옛날에는 영어는 일본보다 통하지 않아서 필기 한자 쪽이 중...
4,韓国旅行,辛いは辛くないって本当,2024,매운 것은 괴롭지 않다는 사실
5,韓国旅行,一回乗り換え駅で改札エラー出た時韓国語で捲し立てられてどうしていいかわからなくてパニクったこ...,2024,한 번 갈아타는 역에서 개찰구 에러가 나왔을 때 한국어로 권설되어 어째서 좋을지 모...


In [ ]:
import deepl
from google.colab import userdata

auth_key = userdata.get('deepl_api_key')  # Replace with your key
translator = deepl.Translator(auth_key)

def translate_text(text):
    try:
        result = translator.translate_text(text, source_lang="JA", target_lang="KO")
        return result.text
    except Exception as e:
        print(f"Error translating text: {text}. Error: {e}")
        return 'error'

df_cleaned["trans_comments"] = df_cleaned["comments"].apply(translate_text)
display(df_cleaned.head())

Error translating text: . Error: text must not be empty
Error translating text: . Error: text must not be empty
Error translating text: . Error: text must not be empty
Error translating text: . Error: text must not be empty


,keywords,comments,years,trans_comments
0,韓国旅行,wifiは韓国の日本語サイトで予約したら半額で借りれる,2024,와이파이를 한국 일본어 사이트에서 예약하면 반값에 대여할 수 있다.
1,韓国旅行,辛くないが辛い辛くないの食べたい時はどうしたらいいんだ,2024,맵지 않은 매운맛을 먹고 싶을 때는 어떻게 하면 좋을까?
3,韓国旅行,韓国も随分変わりましたな昔は英語は日本より通じなくて手書きの漢字の方が中年の方との会話には通...,2024,"한국도 많이 변했네요, 예전에는 영어가 일본보다 통하지 않아 손글씨로 쓴 한자가 중..."
4,韓国旅行,辛いは辛くないって本当,2024,매운 것은 맵지 않다는 사실
5,韓国旅行,一回乗り換え駅で改札エラー出た時韓国語で捲し立てられてどうしていいかわからなくてパニクったこ...,2024,한번은 환승역에서 개찰구 에러가 났을 때 한국어로 호통을 쳐서 어떻게 해야 할지 몰...


In [ ]:
# csv output
df_cleaned.to_csv('japan_youtube_comments.csv', index = False)